In [2]:
# ключевые слова
KEYWORDS = ['python', 'парсинг']

In [3]:
# импорт библиотек
import requests
from bs4 import BeautifulSoup

In [4]:
# страница с ключевыми словами

# объединим все ключевые слова через +
q = '+'.join(KEYWORDS)

#создадим запрос с ключевыми словами
url = 'https://habr.com/ru/search/page1/?q={}&target_type=posts&flow=&order_by=relevance'.format(q)

req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [5]:
# узнаем кол-во страниц, разделив кол-во всех публикаций
# на кол-во публикаций на страницу 
counter_class = "tabs-menu__item-counter tabs-menu__item-counter_total"
counter = soup.find_all('span', class_=counter_class)[0]
pub_count = int(counter.text)

pages = pub_count // 20 + 1 if pub_count % 20 != 0 else int(pub_count / 20)
pages_url = [url.replace('page1', 'page{}'.format(x)) for x in range(1, pages+1)]

# все страницы в этом массиве
pages_url

['https://habr.com/ru/search/page1/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page2/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page3/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page4/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page5/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page6/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page7/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page8/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page9/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/ru/search/page10/?q=python+парсинг&target_type=posts&flow=&order_by=relevance',
 'https://habr.com/

In [ ]:
from time import sleep

# заведем словарь с ключевыми столбцами для парсинга в датафрейм
post_info = {
    "id": [],
    "Заголовок": [],
    "Описание": [],
    "Ссылка": [],
    "Дата": []
}

# переберем все странцы
for page_url in pages_url:
    page_req = requests.get(page_url)
    page_soup = BeautifulSoup(page_req.text, 'html.parser')
    posts = page_soup.find_all('article', class_='post')

    # поищем в постах информацию о посте
    for post in posts:
        post_id = post.parent.attrs.get('id')
        # если идентификатор не найден, это что-то странное, пропускаем
        if not post_id:
            continue
        
        # определим ключевые поля
        post_id = int(post_id.split('_')[-1])
        post_text = post.find('div', class_='post__text').text.replace('\n','').replace('\r', '').strip()
        title = post.find_all('a', class_="post__title_link")[0].text.strip()
        pub_time = post.find('span', class_="post__time").text
        
        # добавим поля в соответствующие столбцы
        post_info["id"].append(post_id)
        post_info["Описание"].append(post_text)
        post_info["Заголовок"].append(title)
        post_info["Ссылка"].append("https://habr.com/ru/post/{}/".format(post_id))
        post_info["Дата"].append(pub_time)

In [28]:
import pandas as pd
post_info_df = pd.DataFrame.from_dict(post_info)
post_info_df

,id,Заголовок,Описание,Ссылка,Дата
0,121815,Парсинг на Pуthon. Как собрать архив Голубятен,... на языке Python. Скрипт выполняет парсинг ...,https://habr.com/ru/post/121815/,14 июня 2011 в 19:24
1,521646,Как выбрать решение для парсинга сайтов: класс...,... Визуальное программирование парсинга; Пар...,https://habr.com/ru/post/521646/,1 октября 2020 в 17:17
2,512208,Как проанализировать рынок фотостудий с помощь...,... с SQLite через python используем библиотек...,https://habr.com/ru/post/512208/,31 июля 2020 в 10:00
3,252379,Веб-парсинг на Ruby,... языка программирования Ruby. Парсинг меня...,https://habr.com/ru/post/252379/,6 марта 2015 в 17:40
4,241670,Вглубь Pyparsing: парсим единицы измерения на ...,... парсера для степени. Парсинг произвольного...,https://habr.com/ru/post/241670/,28 октября 2014 в 10:00
...,...,...,...,...,...
626,517460,Взаимодействие с Check Point SandBlast через API,"... , удобный для последующего парсинга параме...",https://habr.com/ru/post/517460/,11 сентября 2020 в 12:21
627,519016,Loki 1.8: досье на молодой и подающий надежды ...,... необходимая информация для парсинга сетев...,https://habr.com/ru/post/519016/,16 сентября 2020 в 11:42
628,522648,RubyGems Digest 2020·09,"... — библиотеки для токенизации, парсинга и ...",https://habr.com/ru/post/522648/,9 октября 2020 в 19:52
629,522792,# Вышел релиз GitLab 13.4 с хранилищем HashiCo...,... значение.Документация по парсингу покрыти...,https://habr.com/ru/post/522792/,9 октября 2020 в 18:11
